<a href="https://colab.research.google.com/github/TeradaZenichi/dgm-2023.2/blob/main/projetos/PVGEN/Code/Normalizing-Flow-Model/Hyperparameter_Optimization-Normalizing_Flows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Results

Results of a previous sweep run can be found at: https://wandb.ai/cristianmaza/Normalizing-Flows-Hyperparameter-Optimization/sweeps/at2gekk4?workspace=user-cristianmaza

# Hyperparameter optimization

## Installation

In [1]:
%cd /content
!git clone https://github.com/TeradaZenichi/dgm-2023.2/
%cd /content/dgm-2023.2/projetos/PVGEN/notebooks/
!pip install -r requirements-colab.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.0/190.0 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.0/241.0 kB 20.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 8.6 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8791 sha256=28926485ffb8c35db23415e07403bc6cf9809dcda7cdb9cf513b356d9d43d648
  Stored in directory: /root/.cache/pip/wheels/e7/f3/22/152153d6eb222ee7a56ff8617d80ee5207207a8c00a7aab794
Successfully built pathtools
/content
Cloning into 'dgm-2023.2'...
remote: Enumerating objects: 208, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (130/130), done.
remote: Total 208 (delta 15), reused 137 (delta 13), pack-reused 58
Receiving objects: 100% (208/208), 29.89 MiB | 13.72 MiB/s, done.
Resolving deltas: 100% (24/24), 

## Load data

In [2]:
import torch
from GEFcom2014 import load_data
import os
from models import *
import numpy as np
import pandas as pd

samples_per_day = 24

# csv file in Normalizing-Flow-Model directory
dataset_dir = '/content/dgm-2023.2/projetos/PVGEN/src/data/Treinamento-Unicamp/unicamp-database-fixed.csv'

data = load_data(path_name=dataset_dir, test_size=200, random_state=0, samples_per_day=samples_per_day)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using gpu: %s ' % torch.cuda.is_available())
gpu = True  # put False to use CPU
print('Using gpu: %s ' % torch.cuda.is_available())
if gpu:
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

dir_path = 'export/multi_nfs/'
if not os.path.isdir(dir_path):  # test if directory exist
    os.makedirs(dir_path)

28416
1184
Indices where PV is always 0: [ 0  1  2  3  4  5 21 22 23]
#LS 1084 days #VS 50 days # TEST 50 days
Using gpu: True 
Using gpu: True 


## Build sets for Learning, Validation and Testing

In [3]:
ylim_loss = [-50, 50]
ymax_plf = 2
ylim_crps = [0, 10]
nb_zones = 1
indices = data[6]

df_x_LS = data[0].copy()
df_y_LS = data[1].copy()
df_x_VS = data[2].copy()
df_y_VS = data[3].copy()
df_x_TEST = data[4].copy()
df_y_TEST = data[5].copy()

nb_days_LS = len(df_y_LS)
nb_days_VS = len(df_y_VS)
nb_days_TEST = len(df_y_TEST)
print('#LS %s days #VS %s days # TEST %s days' % (nb_days_LS/nb_zones, nb_days_VS/nb_zones, nb_days_TEST/nb_zones))


#LS 1084.0 days #VS 50.0 days # TEST 50.0 days


##  Scale sets

In [4]:
from GEFcom2014.models import scale_data_multi, plot_loss

# WARNING: use the scaler fitted on the TRAIN LS SET !!!!
x_LS_scaled, y_LS_scaled, x_VS_scaled, y_VS_scaled, x_TEST_scaled, y_TEST_scaled, y_LS_scaler = scale_data_multi(x_LS=df_x_LS.values, y_LS=df_y_LS.values, x_VS=df_x_VS.values, y_VS=df_y_VS.values, x_TEST=df_x_TEST.values, y_TEST=df_y_TEST.values)

non_null_indexes = list(np.delete(np.asarray([i for i in range(24)]), indices))

# Rebuilt the PV observations with the removed time periods
df_y_TEST.columns = non_null_indexes
for i in indices:
    df_y_TEST[i] = 0
df_y_TEST = df_y_TEST.sort_index(axis=1)

df_y_VS.columns = non_null_indexes
for i in indices:
    df_y_VS[i] = 0
df_y_VS = df_y_VS.sort_index(axis=1)

## Set number of experiments

In [5]:
n_s = 100
N_q = 99

# Set the torch seed for result reproducibility
torch_seed = 0
torch.manual_seed(torch_seed)

## Set the Sweep configuration

In [6]:
import wandb

sweep_config = {
    'method': 'bayes', # or 'random'
    'metric': {
      'name': 'rmse',
      'goal': 'minimize'
    },
    'parameters': {
        'model_type': {
            'values': ['UMNN', 'AN']
        },
        'nb_epoch': {
            'values': [100, 200, 300, 400, 500]
        },
        'nb_steps': {
            'min': 1,
            'max': 10
        },
        'nb_layers': {
            'min': 1,
            'max': 10
        },
        'nb_neurons': {
            'min': 1,
            'max': 10
        }
    }
}

sweep_id = wandb.sweep(sweep_config, project="Normalizing-Flows-Hyperparameter-Optimization")

## Set function to calculate RMSE

In [7]:
from GEFcom2014.models.NFs import build_nfs_scenarios
from GEFcom2014.utils import read_file
import math
import os
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error

def compute_reliability(y_true: np.array, y_quantile: np.array, tag: str = 'pv'):
    """
    Compute averaged reliability score per day over all quantiles.
    :param y_true: true values (n_periods, ).
    :param y_quantile: quantiles (n_periods, n_quantiles).
    :return: PLF array of shape (n_quantiles,)
    """
    nb_q = y_quantile[0].shape[0]

    aq = []
    if tag == 'pv' or tag == 'load':
        # WARNING REMOVE TIME PERIODS WHERE PV GENERATION IS 0 during night hours !!!!
        # indices where PV generation is 0 at day d
        indices = np.where(y_true == 0)[0]
        y_true = np.delete(y_true, indices).copy()
        y_quantile = np.delete(y_quantile, indices, axis=0).copy()

    nb_periods = len(y_true)
    for q in range(0, nb_q):
        aq.append(sum(y_true < y_quantile[:, q]) / nb_periods)

    return 100 * np.asarray(aq)


def calculate_rmse(best_flow, config, n_s, N_q: int, df_y_VS: pd.DataFrame,
                             df_y_TEST: pd.DataFrame, name: str, ymax_plf: float, ylim_crps: list,
                             nb_zones: int, tag: str = 'pv'):
    """
    Compute quantiles from scenarios.
    Compute metrics:
    - PLF
    - reliability diagrams
    - CRPS
    - plots of scenarios & quantiles

    #VS or #TEST = number of days of VS or TEST sets

    :param s_VS: array (#VS*24, n_s)
    :param s_TEST: array (#TEST*24, n_s)
    :param N_q: number of quantiles/percentiles.
    :param df_y_VS: VS true values into a pd.DataFrame with shape (#VS, 24)
    :param df_y_TEST: TEST true values into a pd.DataFrame with shape (#TEST, 24)
    :param name: name to export plots
    :param tag: pv, wind, load
    :param nb_zones: number of zones (PV 3, wind 10, load 1)
    """
    #
    # 0. Generate scenarios
    #

    max_power = 1

    # Scenarios are generated into a dict of length nb days (#VS or # TEST sizes)
    # Each day of the dict is an array of shape (n_scenarios, 24)

    s_TEST = build_nfs_scenarios(n_s=n_s, x=x_TEST_scaled,
                                  y_scaler=y_LS_scaler, flow=best_flow,
                                  conditioner_args=config['conditioner_args'], max=max_power, gpu=gpu, non_null_indexes=non_null_indexes, samples_per_day=samples_per_day)
    s_VS = build_nfs_scenarios(n_s=n_s, x=x_VS_scaled,
                                y_scaler=y_LS_scaler, flow=best_flow,
                                conditioner_args=config['conditioner_args'], max=max_power, gpu=gpu, non_null_indexes=non_null_indexes, samples_per_day=samples_per_day)


    # --------------------------------------------------------------------------------------------------------------
    # 1. Generate quantiles from scenarios
    # --------------------------------------------------------------------------------------------------------------

    q_set = [i / (N_q + 1) for i in range(1, N_q + 1)]
    # Quantiles are generated into an array of shape (n_day*24, N_q)
    q_TEST = np.quantile(s_TEST, q=q_set, axis=1).transpose()
    q_VS = np.quantile(s_VS, q=q_set, axis=1).transpose()

    # --------------------------------------------------------------------------------------------------------------
    # 3. Reliability diagram
    # --------------------------------------------------------------------------------------------------------------
    nb_days_VS = len(df_y_VS)
    nb_days_TEST = len(df_y_TEST)

    aq_TEST = compute_reliability(y_true=df_y_TEST.values.reshape(-1), y_quantile=q_TEST, tag=tag)
    aq_VS = compute_reliability(y_true=df_y_VS.values.reshape(-1), y_quantile=q_VS, tag=tag)

    rmse_TEST = math.sqrt(mean_squared_error(y_true=np.array(q_set) * 100, y_pred=aq_TEST))
    rmse_VS = math.sqrt(mean_squared_error(y_true=np.array(q_set) * 100, y_pred=aq_VS))
    print('RMSE TEST %.2f VS %.2f' % (rmse_TEST, rmse_VS))

    return rmse_TEST

## Set Training function

In [9]:
import wandb
from GEFcom2014.models.NFs import fit_NF
from GEFcom2014.utils import dump_file


def train():
    # Initialize a new wandb run
    with wandb.init() as run:
        # Each hyperparameter is available as an attribute of your run object
        config = run.config

        # Define the model configurations inside the train function
        cf_UMNN_M = {
            'name': 'UMNN_M_1',
            'nb_steps': config.nb_steps,
            'nb_layers': config.nb_layers,
            'nb_neurons': config.nb_neurons,
            'out_size': 20,
            'weight_decay': 5e-4,
            'learning_rate': 1e-4,
            'conditioner_type': 'AutoregressiveConditioner',
            'normalizer_type': 'MonotonicNormalizer',
        }

        cf_AN_M = {
            'name': 'AN_M_1',
            'nb_steps': config.nb_steps,
            'nb_layers': config.nb_layers,
            'nb_neurons': config.nb_neurons,
            'out_size': 2,
            'weight_decay': 1e-2,
            'learning_rate': 1e-4,
            'conditioner_type': 'AutoregressiveConditioner',
            'normalizer_type': 'AffineNormalizer',
        }

        # Select the model based on the `model_type` hyperparameter
        if config.model_type == 'UMNN':
            model_config = cf_UMNN_M
        else:
            model_config = cf_AN_M

        # Train your model

        name = model_config['name'] + '_' + str(torch_seed)
        print(name)

        if model_config['normalizer_type'] == 'AffineNormalizer':
            model_config['conditioner_args'] = {"in_size": y_LS_scaled.shape[1],
                                                "hidden": [model_config['nb_neurons']] * model_config['nb_layers'],
                                                "out_size": 2,
                                                "cond_in": x_LS_scaled.shape[1]}
            normalizer_type = AffineNormalizer
            model_config['normalizer_args'] = {}

        elif model_config['normalizer_type'] == 'MonotonicNormalizer':
            model_config['conditioner_args'] = {"in_size": y_LS_scaled.shape[1],
                                                "hidden": [model_config['nb_neurons']] * model_config['nb_layers'],
                                                "out_size": model_config['out_size'],
                                                "cond_in": x_LS_scaled.shape[1]}
            normalizer_type = MonotonicNormalizer
            model_config['normalizer_args'] = {
                'integrand_net': [model_config['out_size'] * 2,
                                  model_config['out_size'] * 2,
                                  model_config['out_size'] * 2],
                'cond_size': model_config['out_size'],
                'nb_steps': 50,
                'solver': "CCParallel",
                'hot_encoding': True}

        model_config['Adam_args'] = {"lr": model_config['learning_rate'],
                                     "weight_decay": model_config['weight_decay']}

        nfs = buildFCNormalizingFlow(nb_steps=model_config['nb_steps'],
                                      conditioner_type=AutoregressiveConditioner,
                                      conditioner_args=model_config['conditioner_args'],
                                      normalizer_type=normalizer_type,
                                      normalizer_args=model_config['normalizer_args'])
        opt = torch.optim.Adam(nfs.parameters(),
                               lr=model_config['learning_rate'],
                               weight_decay=model_config['weight_decay'])

        print('Fit NFs with %s epochs' % (config.nb_epoch))
        loss, best_flow, last_flow = fit_NF(nb_epoch=config.nb_epoch,
                                            x_LS=x_LS_scaled, y_LS=y_LS_scaled,
                                            x_VS=x_VS_scaled, y_VS=y_VS_scaled,
                                            x_TEST=x_TEST_scaled, y_TEST=y_TEST_scaled,
                                            flow=nfs,
                                            opt=opt, gpu=gpu)
        rmse = calculate_rmse(best_flow, model_config, n_s, N_q=N_q, df_y_VS=df_y_VS, df_y_TEST=df_y_TEST,
                         name=name, ymax_plf=ymax_plf, ylim_crps=ylim_crps, nb_zones=nb_zones)
        print(rmse)
        # Log the RMSE with wandb.log()
        wandb.log({"rmse": rmse})

        # save model
        model_name = 'model_' + wandb.run.name + '_best'
        dump_file(dir=dir_path, name=model_name, file=best_flow)

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: at2gekk4
Sweep URL: https://wandb.ai/cristianmaza/Normalizing-Flows-Hyperparameter-Optimization/sweeps/at2gekk4


## Train models

In [10]:
wandb.agent(sweep_id, train)

wandb: Agent Starting Run: sddbmn0m with config:
wandb: 	model_type: UMNN
wandb: 	nb_epoch: 500
wandb: 	nb_layers: 5
wandb: 	nb_neurons: 2
wandb: 	nb_steps: 3
wandb: Currently logged in as: cristianmaza. Use `wandb login --relogin` to force relogin


UMNN_M_1_0
Fit NFs with 500 epochs
Fitting time_tot 3 min
3 6
Scenario generation time_tot 0.3 min
3 6
Scenario generation time_tot 0.3 min
RMSE TEST 1.49 VS 1.90
1.4921842616451473


rmse,▁
rmse,1.49218


wandb: Agent Starting Run: 0blbb9t1 with config:
wandb: 	model_type: AN
wandb: 	nb_epoch: 100
wandb: 	nb_layers: 2
wandb: 	nb_neurons: 2
wandb: 	nb_steps: 1


AN_M_1_0
Fit NFs with 100 epochs
Fitting time_tot 0 min
3 6
Scenario generation time_tot 0.0 min
3 6
Scenario generation time_tot 0.0 min
RMSE TEST 7.23 VS 7.45
7.233987040118358


...

UMNN_M_1_0
Fit NFs with 500 epochs
Fitting time_tot 10 min
3 6
Scenario generation time_tot 1.0 min
3 6
Scenario generation time_tot 1.0 min
RMSE TEST 1.81 VS 1.78
1.8076119030731808


rmse,▁
rmse,1.80761


wandb: Agent Starting Run: zigv836c with config:
wandb: 	model_type: AN
wandb: 	nb_epoch: 500
wandb: 	nb_layers: 4
wandb: 	nb_neurons: 2
wandb: 	nb_steps: 3


AN_M_1_0
Fit NFs with 500 epochs


wandb: Ctrl + C detected. Stopping sweep.
